In [1]:
class CFG:
    exps = ['026', 'deberta-v3-large-e1_exp032']
    n_fold = 4
    debug=False
    seed = 42

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-stacking'
    dataset_version = 'exp' + '-'.join(CFG.exps)
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [2]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [3]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
Mounted at /content/drive
     |████████████████████████████████| 58 kB 4.0 MB/s 
CPU times: user 2.57 s, sys: 502 ms, total: 3.08 s
Wall time: 28.6 s


In [4]:
import os
import random
import pickle
from pathlib import Path
from psutil import cpu_count
import warnings
warnings.filterwarnings("ignore")

os.system('python -m pip install -U pickle5')
import pickle5

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
from tqdm.auto import tqdm
tqdm.pandas()
os.system('python -m pip install -U lightgbm')
import lightgbm as lgb
print(f"lightgbm.__version__: {lgb.__version__}")
os.system('python -m pip install optuna')
import optuna.integration.lightgbm as lgb_optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
print(f"torch.__version__: {torch.__version__}")

os.system('python -m pip install sentencepiece')
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

lightgbm.__version__: 3.3.2
torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [5]:
input_dir = Path('../input/us-patent-phrase-to-phrase-matching')
output_dir = Path(f"/content/drive/MyDrive/kaggle/PPPM/output/stacking{'_'.join(CFG.exps)}")
output_dir.mkdir(exist_ok=True)

ColabConfig.dataset_dir = str(output_dir)

In [6]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=output_dir / 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [7]:
# 特徴量エンジニアリング
def tok_len(sentence, tokenizer):
    return len(tokenizer.tokenize(sentence))

def tokenizer_feature_engineering(df, tokenizer):
    def _tok_len(sentence):
        return tok_len(sentence, tokenizer)
    
    basecols = df.columns

    df['anchor_tok_len'] = df['anchor'].map(_tok_len)
    df['target_tok_len'] = df['target'].map(_tok_len)
    df['context_tok_len'] = df['context_text'].map(_tok_len)
    df['input_len'] = df['anchor_tok_len'] + df['target_tok_len'] + df['context_tok_len'] + 4

    df['len_anc_tgt_diff'] = df['anchor_tok_len'] - df['target_tok_len']
    df['len_anc_tgt_div'] = df['anchor_tok_len'] / df['target_tok_len']

    df['len_anc_cnt_diff'] = df['anchor_tok_len'] - df['context_tok_len']
    df['len_anc_cnt_div'] = df['anchor_tok_len'] / df['context_tok_len']

    df['len_tgt_cnt_diff'] = df['target_tok_len'] - df['context_tok_len']
    df['len_tgt_cnt_div'] = df['target_tok_len'] / df['context_tok_len']

    # 作成した特徴量とidカラムだけ返す
    usecols = [col for col in df.columns if col not in basecols]
    usecols.append('id')
    return df[usecols]

In [8]:
def make_ensemble_datasets(cfg):
    cv_scores = []
    train = pd.read_csv(input_dir / 'train.csv')

    for exp in cfg.exps:
        oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{exp}')
        if exp in ['deberta-v3-large-e1_exp032', 'PatentSBERTa-exp035']:
            oof_df = pd.read_csv(oof_dir / 'oof_df.csv')
        else:
            oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
        tokenizer = AutoTokenizer.from_pretrained(oof_dir / 'tokenizer')
        
        # 特徴量と予測スコアは'pred_000'のようなカラム名にする
        feature_df = tokenizer_feature_engineering(oof_df, tokenizer)
        rename_dict = {col: col+'_'+exp for col in feature_df.columns if col != 'id'}
        feature_df = feature_df.rename(columns=rename_dict)
        
        train = train.merge(feature_df, on='id', how='left')
        train = train.merge(oof_df[['id', 'pred']].rename(columns={'pred': f'pred_{exp}'}), 
                            on='id', how='left')

        cv_scores.append(get_score(train['score'], train[f'pred_{exp}']))
    
    print(f'max score: {max(cv_scores)}')
    print(f'avg score: {sum(cv_scores) / len(cv_scores)}')
    return train

In [9]:
pd.__version__

'1.3.5'

In [10]:
train = make_ensemble_datasets(CFG)
display(train.head())

max score: 0.832151085616115
avg score: 0.8237038950893069


,id,anchor,target,context,score,anchor_tok_len_026,target_tok_len_026,context_tok_len_026,input_len_026,len_anc_tgt_diff_026,len_anc_tgt_div_026,len_anc_cnt_diff_026,len_anc_cnt_div_026,len_tgt_cnt_diff_026,len_tgt_cnt_div_026,pred_026,anchor_tok_len_deberta-v3-large-e1_exp032,target_tok_len_deberta-v3-large-e1_exp032,context_tok_len_deberta-v3-large-e1_exp032,input_len_deberta-v3-large-e1_exp032,len_anc_tgt_diff_deberta-v3-large-e1_exp032,len_anc_tgt_div_deberta-v3-large-e1_exp032,len_anc_cnt_diff_deberta-v3-large-e1_exp032,len_anc_cnt_div_deberta-v3-large-e1_exp032,len_tgt_cnt_diff_deberta-v3-large-e1_exp032,len_tgt_cnt_div_deberta-v3-large-e1_exp032,pred_deberta-v3-large-e1_exp032
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0.440476,1,3,29,37,-2,0.333333,-28,0.034483,-26,0.103448,0.358203
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0.641294,1,5,29,39,-4,0.200000,-28,0.034483,-24,0.172414,0.610718
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.247038,1,2,29,36,-1,0.500000,-28,0.034483,-27,0.068966,0.258612
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.494176,1,2,29,36,-1,0.500000,-28,0.034483,-27,0.068966,0.401216
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.011766,1,2,29,36,-1,0.500000,-28,0.034483,-27,0.068966,0.047901


In [11]:
# foldに分ける
# credits to: https://www.kaggle.com/code/abhishek/creating-folds-properly-hopefully-p

!pip install -q iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

dfx = pd.get_dummies(train, columns=["score"]).groupby(["anchor"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("score_") or c == "anchor"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "anchor"]
dfx_labels = dfx[labels]
dfx["fold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    print(len(trn_), len(val_))
    dfx.loc[val_, "fold"] = fold

train = train.merge(dfx[["anchor", "fold"]], on="anchor", how="left")
print(train.fold.value_counts())

550 183
549 184
550 183
550 183
3    9622
0    9379
1    8860
2    8612
Name: fold, dtype: int64


In [12]:
# カテゴリ変数の変換
train['section'] = train['context'].map(lambda s: s[0])
cat_cols = ['section']
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[f'{c}_encoded'] = le.transform(train[c])
    with open(output_dir / f'{c}_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)

In [13]:
print(train.columns)

Index(['id', 'anchor', 'target', 'context', 'score', 'anchor_tok_len_026', 'target_tok_len_026', 'context_tok_len_026', 'input_len_026', 'len_anc_tgt_diff_026', 'len_anc_tgt_div_026', 'len_anc_cnt_diff_026', 'len_anc_cnt_div_026', 'len_tgt_cnt_diff_026', 'len_tgt_cnt_div_026', 'pred_026', 'anchor_tok_len_deberta-v3-large-e1_exp032', 'target_tok_len_deberta-v3-large-e1_exp032', 'context_tok_len_deberta-v3-large-e1_exp032', 'input_len_deberta-v3-large-e1_exp032', 'len_anc_tgt_diff_deberta-v3-large-e1_exp032', 'len_anc_tgt_div_deberta-v3-large-e1_exp032', 'len_anc_cnt_diff_deberta-v3-large-e1_exp032', 'len_anc_cnt_div_deberta-v3-large-e1_exp032', 'len_tgt_cnt_diff_deberta-v3-large-e1_exp032', 'len_tgt_cnt_div_deberta-v3-large-e1_exp032', 'pred_deberta-v3-large-e1_exp032', 'fold', 'section', 'section_encoded'], dtype='object')


In [14]:
excluded_columns = ['id', 'anchor', 'target', 'context', 'score', 'fold', 'section']
cols = [col for col in train.columns if col not in excluded_columns]
with open(output_dir / 'cols.pkl', 'wb') as f:
    pickle.dump(cols, f)
print(cols)

['anchor_tok_len_026', 'target_tok_len_026', 'context_tok_len_026', 'input_len_026', 'len_anc_tgt_diff_026', 'len_anc_tgt_div_026', 'len_anc_cnt_diff_026', 'len_anc_cnt_div_026', 'len_tgt_cnt_diff_026', 'len_tgt_cnt_div_026', 'pred_026', 'anchor_tok_len_deberta-v3-large-e1_exp032', 'target_tok_len_deberta-v3-large-e1_exp032', 'context_tok_len_deberta-v3-large-e1_exp032', 'input_len_deberta-v3-large-e1_exp032', 'len_anc_tgt_diff_deberta-v3-large-e1_exp032', 'len_anc_tgt_div_deberta-v3-large-e1_exp032', 'len_anc_cnt_diff_deberta-v3-large-e1_exp032', 'len_anc_cnt_div_deberta-v3-large-e1_exp032', 'len_tgt_cnt_diff_deberta-v3-large-e1_exp032', 'len_tgt_cnt_div_deberta-v3-large-e1_exp032', 'pred_deberta-v3-large-e1_exp032', 'section_encoded']


In [15]:
params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'num_iterations': 1000,
    'learning_rate': 0.01,
    # 'num_leaves': 20,  # 31
    'num_threads': cpu_count(logical=False),
    'seed': CFG.seed,

    # 'max_depth': 3,  # -1
    # 'min_data_in_leaf': 100,  # 20
    'verbosity': -1,  # 1

    'metric': 'l2'
}

In [16]:
def tuning_params(df, params):
    anchors = df['anchor'].unique()
    rng = np.random.default_rng(seed=71)
    rng.shuffle(anchors)
    val_prop = 0.25  # バリデーションデータの割合。4foldから0.25
    val_size = int(len(anchors)*val_prop)
    val_anchors = anchors[:val_size]
    is_val = np.isin(df['anchor'], val_anchors)
    idx = np.arange(len(df))
    val_idx = idx[is_val]
    tr_idx = idx[~is_val]
    print(f'train: {len(tr_idx)}, val: {len(val_idx)}')

    tr_df = df.iloc[tr_idx]
    val_df = df.iloc[val_idx]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)
    
    model = lgb_optuna.train(
        params,
        tr_ds, 
        valid_sets=[tr_ds, val_ds],
        callbacks=[lgb.early_stopping(stopping_rounds=100),
                    lgb.log_evaluation(period=100)],
        verbosity=-1,
    )

    best_params = model.params

    return best_params

In [ ]:
best_params = tuning_params(train, params)
display(best_params)

train: 27261, val: 9212


[I 2022-06-20 09:19:19,531] A new study created in memory with name: no-name-334595c9-4581-4b7d-adfa-d12d566d0445
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l2: 0.0662933	valid_1's l2: 0.0643284
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 0.0653651	valid_1's l2: 0.0634327
[3]	valid_0's l2: 0.0644558	valid_1's l2: 0.0625555
[4]	valid_0's l2: 0.0635645	valid_1's l2: 0.0616964
[5]	valid_0's l2: 0.0626905	valid_1's l2: 0.060852
[6]	valid_0's l2: 0.0618343	valid_1's l2: 0.0600252
[7]	valid_0's l2: 0.0609953	valid_1's l2: 0.0592157
[8]	valid_0's l2: 0.0601721	valid_1's l2: 0.0584213
[9]	valid_0's l2: 0.0593729	valid_1's l2: 0.0576468
[10]	valid_0's l2: 0.0585822	valid_1's l2: 0.0568838
[11]	valid_0's l2: 0.0578418	valid_1's l2: 0.0561734
[12]	valid_0's l2: 0.057081	valid_1's l2: 0.0554418
[13]	valid_0's l2: 0.0563354	valid_1's l2: 0.054724
[14]	valid_0's l2: 0.055604	valid_1's l2: 0.05402
[15]	valid_0's l2: 0.054888	valid_1's l2: 0.053332
[16]	valid_0's l2: 0.0541855	valid_1's l2: 0.0526548
[17]	valid_0's l2: 0.0534972	valid_1's l2: 0.0519929
[18]	valid_0's l2: 0.0528221	valid_1's l2: 0.0513437
[

feature_fraction, val_score: 0.019794:  14%|#4        | 1/7 [00:06<00:40,  6.73s/it]

[1]	valid_0's l2: 0.0663024	valid_1's l2: 0.0643318
[2]	valid_0's l2: 0.0653745	valid_1's l2: 0.0634347
[3]	valid_0's l2: 0.0644738	valid_1's l2: 0.0625619
[4]	valid_0's l2: 0.0635913	valid_1's l2: 0.0617061
[5]	valid_0's l2: 0.0627573	valid_1's l2: 0.060904
[6]	valid_0's l2: 0.0626595	valid_1's l2: 0.0608342
[7]	valid_0's l2: 0.0618438	valid_1's l2: 0.0600505
[8]	valid_0's l2: 0.0610117	valid_1's l2: 0.0592434
[9]	valid_0's l2: 0.0609188	valid_1's l2: 0.0591769
[10]	valid_0's l2: 0.0600966	valid_1's l2: 0.0583841
[11]	valid_0's l2: 0.0593288	valid_1's l2: 0.0576463
[12]	valid_0's l2: 0.0585453	valid_1's l2: 0.0568862
[13]	valid_0's l2: 0.0584515	valid_1's l2: 0.0568157
[14]	valid_0's l2: 0.057715	valid_1's l2: 0.0561074
[15]	valid_0's l2: 0.0569556	valid_1's l2: 0.0553755
[16]	valid_0's l2: 0.0568739	valid_1's l2: 0.0553167
[17]	valid_0's l2: 0.0567942	valid_1's l2: 0.055258
[18]	valid_0's l2: 0.0567141	valid_1's l2: 0.0551997
[19]	valid_0's l2: 0.0566352	valid_1's l2: 0.0551443
[20]	

...

[1]	valid_0's l2: 0.0663038	valid_1's l2: 0.0643323
[2]	valid_0's l2: 0.0653782	valid_1's l2: 0.0634367
[3]	valid_0's l2: 0.0644709	valid_1's l2: 0.0625592
[4]	valid_0's l2: 0.0635893	valid_1's l2: 0.0617033
[5]	valid_0's l2: 0.0627172	valid_1's l2: 0.0608605
[6]	valid_0's l2: 0.0618627	valid_1's l2: 0.0600345
[7]	valid_0's l2: 0.0610626	valid_1's l2: 0.059266
[8]	valid_0's l2: 0.0602402	valid_1's l2: 0.0584724
[9]	valid_0's l2: 0.0594411	valid_1's l2: 0.0576968
[10]	valid_0's l2: 0.0586508	valid_1's l2: 0.0569345
[11]	valid_0's l2: 0.057911	valid_1's l2: 0.0562238
[12]	valid_0's l2: 0.0571508	valid_1's l2: 0.0554915
[13]	valid_0's l2: 0.0564399	valid_1's l2: 0.054809
[14]	valid_0's l2: 0.0557431	valid_1's l2: 0.0541402
[15]	valid_0's l2: 0.0550257	valid_1's l2: 0.0534489
[16]	valid_0's l2: 0.0543274	valid_1's l2: 0.0527722
[17]	valid_0's l2: 0.0542459	valid_1's l2: 0.0527129
[18]	valid_0's l2: 0.0535905	valid_1's l2: 0.0520846
[19]	valid_0's l2: 0.0529483	valid_1's l2: 0.0514678
[20]	

num_leaves, val_score: 0.019721:  50%|#####     | 10/20 [01:11<01:13,  7.38s/it]

[675]	valid_0's l2: 0.0180615	valid_1's l2: 0.0197786
[676]	valid_0's l2: 0.0180603	valid_1's l2: 0.0197785
[677]	valid_0's l2: 0.0180589	valid_1's l2: 0.0197782
[678]	valid_0's l2: 0.0180572	valid_1's l2: 0.019778
[679]	valid_0's l2: 0.0180554	valid_1's l2: 0.0197781
[680]	valid_0's l2: 0.0180536	valid_1's l2: 0.0197776
[681]	valid_0's l2: 0.0180524	valid_1's l2: 0.0197773
[682]	valid_0's l2: 0.0180509	valid_1's l2: 0.0197774
Early stopping, best iteration is:
[582]	valid_0's l2: 0.018959	valid_1's l2: 0.0197208
[1]	valid_0's l2: 0.0662798	valid_1's l2: 0.0643302
[2]	valid_0's l2: 0.0653242	valid_1's l2: 0.0634293
[3]	valid_0's l2: 0.0643883	valid_1's l2: 0.0625466
[4]	valid_0's l2: 0.0634836	valid_1's l2: 0.0616888
[5]	valid_0's l2: 0.0625833	valid_1's l2: 0.0608412
[6]	valid_0's l2: 0.0616996	valid_1's l2: 0.0600096
[7]	valid_0's l2: 0.0608773	valid_1's l2: 0.0592374
[8]	valid_0's l2: 0.0600274	valid_1's l2: 0.0584399
[9]	valid_0's l2: 0.0592065	valid_1's l2: 0.0576637
[10]	valid_0'

num_leaves, val_score: 0.019721:  55%|#####5    | 11/20 [01:24<01:22,  9.17s/it]

[1]	valid_0's l2: 0.0662946	valid_1's l2: 0.064332
[2]	valid_0's l2: 0.0653565	valid_1's l2: 0.0634318
[3]	valid_0's l2: 0.0644373	valid_1's l2: 0.0625496
[4]	valid_0's l2: 0.0635469	valid_1's l2: 0.0616941
[5]	valid_0's l2: 0.0626632	valid_1's l2: 0.060847
[6]	valid_0's l2: 0.0617968	valid_1's l2: 0.0600159
[7]	valid_0's l2: 0.060989	valid_1's l2: 0.0592452
[8]	valid_0's l2: 0.0601557	valid_1's l2: 0.0584481
[9]	valid_0's l2: 0.0593483	valid_1's l2: 0.0576744
[10]	valid_0's l2: 0.0585473	valid_1's l2: 0.0569082
[11]	valid_0's l2: 0.0577992	valid_1's l2: 0.056196
[12]	valid_0's l2: 0.0570287	valid_1's l2: 0.0554603
[13]	valid_0's l2: 0.0563105	valid_1's l2: 0.0547763
[14]	valid_0's l2: 0.0556062	valid_1's l2: 0.0541068
[15]	valid_0's l2: 0.0548786	valid_1's l2: 0.0534137
[16]	valid_0's l2: 0.0541726	valid_1's l2: 0.0527375
[17]	valid_0's l2: 0.054079	valid_1's l2: 0.0526752
[18]	valid_0's l2: 0.053417	valid_1's l2: 0.0520467
[19]	valid_0's l2: 0.0527671	valid_1's l2: 0.0514286
[20]	val

In [ ]:
def prepare_datasets(df, fold):
    tr_df = df[df['fold']!=fold]
    val_df = df[df['fold']==fold]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)

    return tr_ds, val_ds


def train_fn(df, params):
    preds = []

    for fold in df['fold'].unique():
        tr_ds, val_ds = prepare_datasets(df, fold)
        
        model = lgb.train(
            params, 
            tr_ds, 
            valid_sets=[tr_ds, val_ds],
            callbacks=[lgb.early_stopping(stopping_rounds=100),
                       lgb.log_evaluation(period=100)]
        )

        pred = df[df['fold']==fold].copy()
        pred['pred_lgb'] = model.predict(pred[cols], num_iteration=model.best_iteration)
        preds.append(pred)

        model.save_model(output_dir / f'lgb_fold{fold}.txt', num_iteration=model.best_iteration)

    return pd.concat(preds, axis=0, ignore_index=False)

In [ ]:
pred = train_fn(train, best_params)

In [ ]:
importances = []
for fold in train['fold'].unique():
    importances.append(
        lgb.Booster(model_file=output_dir / f'lgb_fold{fold}.txt').feature_importance(importance_type='gain'))
importance_df = pd.DataFrame({
    'feature': cols,
    'importance': np.mean(importances, axis=0)
    })
importance_df.sort_values('importance', ascending=False)

In [ ]:
# optunaのbest params
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

In [ ]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

ID = 'hanejiyuto'
DATASET_ID = ColabConfig.dataset_name + '-' + ColabConfig.dataset_version
UPLOAD_DIR = ColabConfig.dataset_dir
VERSION_NOTES = ColabConfig.dataset_note

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets create -t -p $UPLOAD_DIR -r tar

def dataset_create_version():  # バージョンアップデート
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    if not os.path.exists('dataset-metadata.json'):
        with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_version(folder=UPLOAD_DIR, version_notes=VERSION_NOTES, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets version -t -p $UPLOAD_DIR -r tar -m $VERSION_NOTES

if CFG.debug:
    pass
elif ColabConfig.dataset_new:
    dataset_create_new()
else:
    dataset_create_version()